In [ ]:
import nibabel as nib
import os
import h5py
from sklearn import linear_model
from scipy import stats
import numpy as np
import json
import glob
#path to data (change to your path)
fmripreppath = '/data/MoL_clean/fmriprep/'

#path to output (change to your path)
prepath = '/data/MoL_clean/preprocessed/'

#list of sessions
sesslist=['ses-01','ses-02','ses-03']

#list of tasks/#runs in each session
tasklist = [[('Loci', 2),('Item',2),('Encode',1),('Retrieve',1)],
            [('Loci', 2),('Item',2),('Encode',1),('Retrieve',1)],
            [('Loci', 2),('Item',2),('Encode',1),('Retrieve',1)]]

#list of subjects
subs= ['sub-%02d' % i for i in range(1,26)]


In [16]:
os.path.exists(os.path.exists(sess_prefix + '_task-' + task[0] + ('_run-%.2d' % offset) + '_desc-confounds_timeseries.tsv'))


In [ ]:
# identify group masks for hippocampus
regional_mask_filenames = sorted([f for f in glob.glob(fmripreppath+'*/ses-01/func/*_ses-01_task-Loci_run-01_space-MNI152NLin2009cAsym_desc-aseg_dseg.nii.gz') ])
regional_mask_filenames = regional_mask_filenames[:10]+regional_mask_filenames[16:]
regional_masks = []
hippo_masks = []
for regional_mask_fname in regional_mask_filenames:
    regional_mask = nib.load(regional_mask_fname).get_fdata()
    regional_masks.append(regional_mask)
    hippo_masks.append(np.where(np.logical_or(regional_mask == 17, regional_mask == 53)))

hippo_group_mask_template = np.zeros(regional_mask.shape)
hippo_group_mask = np.zeros(regional_mask.shape)
for hippo_mask in hippo_masks:
    hippo_group_mask_template[hippo_mask] += 1  
hippo_group_mask[hippo_group_mask_template==25] = 1

anterior_group,posterior_group = hippo_group_mask.copy(),hippo_group_mask.copy()
anterior_group[:,55:,:] = False
posterior_group[:,:55,:] = False
anterior_group = anterior_group.astype(bool)
posterior_group = posterior_group.astype(bool)
hippo_group_mask = hippo_group_mask.astype(bool)

In [117]:

for sub in subs:
    print('Processing subject:', sub)

    for sess_ind, sess in enumerate(sesslist):
        print('session:', sess)

        if sess == '':
            sess_prefix = os.path.join(fmripreppath, sub, sess, 'func', sub)
        else:
            sess_prefix = os.path.join(fmripreppath, sub, sess, 'func', sub + '_' + sess)

        sess_tasknames = []
        for task in tasklist[sess_ind]:
            if task[1] == 1:
                sess_tasknames.append(task[0]+'_run-01')
            else:
                # For run numbers that don't start at zero
                offset = 0
                while not os.path.exists(sess_prefix + '_task-' + task[0] + ('_run-%.2d' % offset) + '_desc-confounds_timeseries.tsv'):
                    print('looping')
                    offset += 1
                    if offset>10:
                        print("something is wrong")
                        break
                for r in range(task[1]):
                    sess_tasknames.append('%s_run-%.2d' % (task[0], r+offset))
        for task_name in sess_tasknames:
            print('task:', task_name)

            task_prefix = sess_prefix + '_task-' + task_name

            D = dict()
            for hem in ['L', 'R', 'Vol']:
                if hem == 'Vol':
                    # Load all timecourses in the 3d brain mask

                    fname =  task_prefix + '_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
                    print('      Loading ', fname)
                    
                    nii_4d = nib.load(fname).get_fdata()

                    mask_fname = task_prefix + '_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'
                    mask_3d = nib.load(mask_fname).get_fdata().astype(bool)
                    
                    regional_mask_fname = task_prefix + '_space-MNI152NLin2009cAsym_desc-aseg_dseg.nii.gz'
                    regional_mask = nib.load(regional_mask_fname).get_fdata()
                    hippo_mask = np.logical_or(regional_mask == 17, regional_mask == 53)
                    anterior,posterior = hippo_mask.copy(),hippo_mask.copy()
                    anterior[:,55:,:] = False
                    posterior[:,:55,:] = False
                    D['anterior_hipp'] = nii_4d[anterior]
                    D['posterior_hipp'] = nii_4d[posterior]
                    D['hippo'] = nii_4d[hippo_mask]
                    D['anterior_hipp_group'] = nii_4d[anterior_group]
                    D['posterior_hipp_group'] = nii_4d[posterior_group]
                    D['hippo_group'] = nii_4d[hippo_group_mask]
                    D[hem] = nii_4d[mask_3d]
                    
                else:
                    # Load all timecourses for one cortical hemisphere

                    fname = task_prefix +'_hemi-'+hem+'_space-fsaverage6_bold.func.gii'
                    print('      Loading ', fname)

                    gi = nib.load(fname)
                    D[hem] = np.column_stack([gi.darrays[t].data for t in range(len(gi.darrays))])


            # Load confound regressors
            conf = np.genfromtxt(task_prefix + '_desc-confounds_timeseries.tsv', names=True)

            conf_json = json.load(open(task_prefix + '_desc-confounds_timeseries.json'))

            # Find first combined compcor regressor
            first_cc = 0
            while True:
                if conf_json['a_comp_cor_%02d' % first_cc]['Mask'] == 'combined':
                    break
                first_cc += 1

            reg = np.column_stack((

                # Motion and motion derivatives
                conf['trans_x'],
                conf['trans_x_derivative1'],
                conf['trans_y'],
                conf['trans_y_derivative1'],
                conf['trans_z'],
                conf['trans_z_derivative1'],
                conf['rot_x'],
                conf['rot_x_derivative1'],
                conf['rot_y'],
                conf['rot_y_derivative1'],
                conf['rot_z'],
                conf['rot_z_derivative1'],
                conf['framewise_displacement'],

                # First six compcor components (white matter + CSF signals)
                conf['a_comp_cor_%02d' % first_cc],
                conf['a_comp_cor_%02d' % (first_cc+1)],
                conf['a_comp_cor_%02d' % (first_cc+2)],
                conf['a_comp_cor_%02d' % (first_cc+3)],
                conf['a_comp_cor_%02d' % (first_cc+4)],
                conf['a_comp_cor_%02d' % (first_cc+5)],

                # Cosine (drift) and motion spikes
                np.column_stack([conf[k] for k in conf.dtype.names if ('cosine' in k) or ('motion_outlier' in k)])))

            # Remove nans, e.g. from framewise_displacement
            reg = np.nan_to_num(reg)

            print('      Cleaning and zscoring')
            for hem in ['L', 'R', 'Vol', 'anterior_hipp','posterior_hipp','hippo','anterior_hipp_group','posterior_hipp_group','hippo_group']:
                # Regress out confounds from data
                regr = linear_model.LinearRegression()
                regr.fit(reg, D[hem].T)
                D[hem] = D[hem] - np.dot(regr.coef_, reg.T) - regr.intercept_[:, np.newaxis]
                # Note 8% of values on cortical surface are NaNs, and the following will therefore throw an error
                D[hem] = stats.zscore(D[hem], axis=1)

            # Save hdf5 file
            if sess == '':
                savepath = os.path.join(prepath, sub + '_' + task_name +  '.h5')
            else:
                savepath = os.path.join(prepath, sub + '_' + sess + '_' + task_name +  '.h5')
            with h5py.File(savepath,'w') as hf:
                grp = hf.create_group(task_name)
                grp.create_dataset('L', data=D['L'])
                grp.create_dataset('R', data=D['R'])
                grp.create_dataset('Vol', data=D['Vol'])
                grp.create_dataset('anterior_hipp', data=D['anterior_hipp'])
                grp.create_dataset('posterior_hipp', data=D['posterior_hipp'])
                grp.create_dataset('hippo', data=D['hippo'])
                grp.create_dataset('anterior_hipp_group', data=D['anterior_hipp_group'])
                grp.create_dataset('posterior_hipp_group', data=D['posterior_hipp_group'])
                grp.create_dataset('hippo_group', data=D['hippo_group'])

                grp.create_dataset('reg',data=reg)
                grp.create_dataset('mask', data=mask_3d)
                grp.create_dataset('hippo_mask', data=hippo_mask)

            print('      saved hdf5 file')
